In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import jn  # Bessel fonksiyonu
from scipy.signal import convolve2d
from scipy.signal import fftconvolve
from aotools.turbulence.phasescreen import ft_phase_screen
from scipy.fftpack import fft2, ifft2
from scipy.stats import norm

In [2]:
#Hubble verisine uygulamak üzere Airy diski PSF'si oluşturan fonksiyon
def airy_disk_psf(size = 512, wavelength = 550e-9, aperture_diameter = 1.5):
    """ Airy disk PSF'si hesaplama (teleskop merceği için). """
    
    # Görüntü merkezi
    center = size // 2
    x, y = np.meshgrid(np.arange(size) - center, np.arange(size) - center)

    # Hipotenüs
    r = np.sqrt(x**2 + y**2)

    # Dalga sayısı k = 2*pi / λ
    k = 2 * np.pi / wavelength

    # Airy disk fonksiyonu oluşturulur. (Bessel fonksiyonu kullanılarak)
    psf = (2 * jn(1, k * r * aperture_diameter / 2) / (k * r * aperture_diameter / 2))**2
    psf[np.isnan(psf)] = 0  # r = 0 olduğunda oluşan NaN'leri düzelt
    
    #Normalize ederek döndürülür.
    return psf / np.sum(psf)  



In [3]:
#Hubble verisine uygulamak üzere AOtools ft_phase_screen() fonksiyonunu kullanarak 
#atmosferik faz ekranı oluşturan fonksiyon. Faz ekranı parametrelerinin dağılımı belirlenirken
#TUG'da ölçülen Fried parametresi değerleri göz önüne alınmıştır.

def atmospheric_phase_screen(size):

    l0 = 1e-3
    delta = 0.001
    L0 = np.random.uniform(10,20)
    r0=np.random.normal(0.04,0.01)
    
    phase_screen = ft_phase_screen(r0=r0,N=size, delta = delta, L0=L0, l0=l0)
    return phase_screen
    

In [ ]:
#Faz ekranını görüntüye uygulayan fonksiyon (Atmosferik Türbülans)
def apply_atmospheric_effects(clean_image: np.ndarray, phase_screen):

    #Temiz görsele ileri Fourier dönüşümü uygulanır. (scipy kütüphanesi)
    image_ft=fft2(clean_image)

    #Faz ekranı uygulanır.
    atmospheric_image = image_ft * np.exp(1j * phase_screen)  
    
    #Ters Fourier dönüşümü uygulanarak döndürlür.
    return np.abs(ifft2(atmospheric_image))


    
#PSF'yi görüntüye uygulayan fonksiyon
    def apply_psf(atmospheric_image:np.ndarray, psf):

    blurred_image = fftconvolve(atmospheric_image, psf, mode='same')
    return blurred_image


In [4]:
def distort(img: np.ndarray, psf):

    ps = atmospheric_phase_screen(size = 512, r0= 0.04, L0=15,delta=0.00001, l0 =1e-3)
    atm_img = apply_atmospheric_effects(img, ps)
    fnl_img = apply_psf(atm_img, psf)
    
    return fnl_img
